In [54]:
import pyproj 
import geopandas as gpd
from shapely.geometry import Polygon
import folium

# this function allows to create pie wedge polygon 
# from start coordinates and azimuth
def sector_create(startLon, startLat, forwardAzimuth, distance=500, width = 60, step = 1):
    lon_list, lat_list = [startLon], [startLat]
    for i in range(-int(width/2), int(width/2), step):
        point = pyproj.Geod(ellps='WGS84').fwd(startLon, startLat, forwardAzimuth + i, distance)
        lon_list.append(point[0])
        lat_list.append(point[1])
    lon_list.append(startLon)
    lat_list.append(startLat)
    geo_polygon = Polygon(zip(lon_list, lat_list))
    return geo_polygon 

# this func allows to get difference between polygons
def diff_sectors(gdf, col1, col2):
    diff = gpd.GeoSeries(gdf[col2]).difference(gpd.GeoSeries(gdf[col1]))
    return diff

In [51]:
poly = gpd.GeoDataFrame({'cellname': ['cell_1', 'cell_2', 'cell_3'],
                        'lat': [45.05, 45.05, 45.05],
                        'lon': [38.94, 38.94, 38.94],
                        'azimuth': [90, 200, 300]})
poly


,cellname,lat,lon,azimuth
0,cell_1,45.05,38.94,90
1,cell_2,45.05,38.94,200
2,cell_3,45.05,38.94,300


In [52]:
poly['g'] = poly.apply(lambda row: sector_create(row['lon'], row['lat'], row['azimuth'] ), axis = 1).set_crs('epsg:4326')
poly
# poly['diff'] = poly.apply(lambda row1: diff_sectors(poly, 'geometry', 'geometry_1' ), axis = 1).set_crs('epsg:4326')
# poly['geometry_1'] = poly['geometry_1'].astype(geo)

,cellname,lat,lon,azimuth,g
0,cell_1,45.05,38.94,90,"POLYGON ((38.94000 45.05000, 38.94550 45.05225..."
1,cell_2,45.05,38.94,200,"POLYGON ((38.94000 45.05000, 38.94110 45.04557..."
2,cell_3,45.05,38.94,300,"POLYGON ((38.94000 45.05000, 38.93365 45.05000..."


In [53]:
m = folium.Map([ 45.05, 38.94], zoom_start=14, tiles='cartodbpositron')
folium.GeoJson(poly['g']).add_to(m)
folium.LatLngPopup().add_to(m)
m